## Problema de mezclas: Mezcla de aceites
---
Una empresa elabora un cierto alimento refinando diferentes tipos de aceite y mezclándolos. Los aceites se clasifican en dos categorías: vegetales (VEG1 y VEG2) y no vegetales (OIL1, OIL2 y OIL3).
Cada uno de estos tipos, vegetal o no vegetal, requiere una línea de producción diferente para el refinado:
* La capacidad máxima de refino es de 250 Tn de aceite vegetal y 300 Tn de no vegetal.
* Se puede asumir que el coste de refino es nulo y que durante el proceso no se producen pérdidas de peso.

Cada tonelada de producto final se vende a un precio de 15 EUR.
La siguiente tabla muestra el coste (por tonelada en EUR) de cada tipo de aceite:

|       | VEG1 | VEG2 | OIL1 | OIL2 | OIL3 |
|-------|------|------|------|------|------|
| Coste |  11  |  12  |  13  |  11  | 11.5 |


Por restricciones tecnológicas, la dureza del producto final debe estar entre 4 y 9 unidades. Se puede asumir que la dureza se mezcla linealmente. Esto es, la dureza final es la media ponderada de la dureza de los distintos aceites utilizados en la mezcla.

La siguiente tabla muestra la dureza de cada tipo de aceite:

|       | VEG1 | VEG2 | OIL1 | OIL2 | OIL3 |
|-------|------|------|------|------|------|
| Dureza|  7.2 |  6.3 |  2.0 |  5.2 |  4.0 |

**Determinar qué cantidad de aceite de cada tipo debe usarse en la mezcla para MAXIMIZAR el beneficio obtenido** (diferencia entre el precio de venta de la mezcla y el precio de compra de los aceites).

In [1]:
import gurobipy as grb

# Creamos el modelo
m = grb.Model("Mezcla de aceites")

Set parameter Username
Set parameter LicenseID to value 2718415
Academic license - for non-commercial use only - expires 2026-10-06


In [ ]:
# Datos
# X1= Cantidad de VEG1, X2= Cantidad de VEG2, X3= Cantidad de OIL1, X4= Cantidad de OIL2, X5= Cantidad de OIL3
x=[1,2,3,4,5]
Selling_price=15
Cost_price=[11,12,13,11,11.5]
Hardness=[7.2,6.3,2,5.2,4]

x=m.addVars()
